## 언론사별 뉴스 크롤링 하기

In [6]:
import sys, os
import requests
import selenium
from selenium import webdriver
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime
import pickle, json, glob, time
from tqdm import tqdm



###### 날짜 저장 ##########
date = str(datetime.now())
date = date[:date.rfind(':')].replace(' ', '_')
date = date.replace(':','시') + '분'

sleep_sec = 0.5


####### 언론사별 본문 위치 태그 파싱 함수 ###########
print('본문 크롤링에 필요한 함수를 로딩하고 있습니다...\n' + '-' * 100)
def crawling_main_text(url):

    req = requests.get(url)
    req.encoding = None
    soup = BeautifulSoup(req.text, 'html.parser')
    
    # 연합뉴스
    if ('://yna' in url) | ('app.yonhapnews' in url): 
        main_article = soup.find('article', {'class':'story-news article'})
        if main_article == None:
            main_article = soup.find('div', {'class' : 'article-txt'})
            
        text = main_article.text
        
    # MBC 
    elif '//imnews.imbc' in url: 
        text = soup.find('div', {'itemprop' : 'articleBody'}).text
        
    # 매일경제(미라클), req.encoding = None 설정 필요
    elif 'mirakle.mk' in url:
        text = soup.find('div', {'class' : 'view_txt'}).text
        
    # 매일경제, req.encoding = None 설정 필요
    elif 'mk.co' in url:
        text = soup.find('div', {'class' : 'art_txt'}).text
        
    # SBS
    elif 'news.sbs' in url:
        text = soup.find('div', {'itemprop' : 'articleBody'}).text
    
    # KBS
    elif 'news.kbs' in url:
        text = soup.find('div', {'id' : 'cont_newstext'}).text
        
    # JTBC
    elif 'news.jtbc' in url:
        text = soup.find('div', {'class' : 'article_content'}).text
        
    # 그 외
    else:
        text == None
        
    return text.replace('\n','').replace('\r','').replace('<br>','').replace('\t','')
    
    
press_nm = 'JTBC'

print('검색할 언론사 : {}'.format(press_nm))


############### 브라우저를 켜고 검색 키워드 입력 ####################
query = input('검색할 키워드  : ')
news_num = int(input('수집 뉴스의 수(숫자만 입력) : '))

print('\n' + '=' * 100 + '\n')

print('브라우저를 실행시킵니다(자동 제어)\n')
chrome_path = 'C:/Users/jhjh3/chromedriver_win32 (1)/chromedriver.exe'
browser = webdriver.Chrome(chrome_path)

news_url = 'https://search.naver.com/search.naver?where=news&query={}'.format(query)
browser.get(news_url)
time.sleep(sleep_sec)


######### 언론사 선택 및 confirm #####################
print('설정한 언론사를 선택합니다.\n')

search_opn_btn = browser.find_element_by_xpath('//a[@class="btn_option _search_option_open_btn"]')
search_opn_btn.click()
time.sleep(sleep_sec)

bx_press = browser.find_element_by_xpath('//div[@role="listbox" and @class="api_group_option_sort _search_option_detail_wrap"]//li[@class="bx press"]')

# 기준 두번 째(언론사 분류순) 클릭하고 오픈하기
press_tablist = bx_press.find_elements_by_xpath('.//div[@role="tablist" and @class="option"]/a')
press_tablist[1].click()
time.sleep(sleep_sec)

# 첫 번째 것(언론사 분류선택)
bx_group = bx_press.find_elements_by_xpath('.//div[@class="api_select_option type_group _category_select_layer"]/div[@class="select_wrap _root"]')[0]

press_kind_bx = bx_group.find_elements_by_xpath('.//div[@class="group_select _list_root"]')[0]
press_kind_btn_list = press_kind_bx.find_elements_by_xpath('.//ul[@role="tablist" and @class="lst_item _ul"]/li/a')


for press_kind_btn in press_kind_btn_list:
    
    # 언론사 종류를 순차적으로 클릭(좌측)
    press_kind_btn.click()
    time.sleep(sleep_sec)
    
    # 언론사선택(우측)
    press_slct_bx = bx_group.find_elements_by_xpath('.//div[@class="group_select _list_root"]')[1]
    # 언론사 선택할 수 있는 클릭 버튼
    press_slct_btn_list = press_slct_bx.find_elements_by_xpath('.//ul[@role="tablist" and @class="lst_item _ul"]/li/a')
    # 언론사 이름들 추출
    press_slct_btn_list_nm = [psl.text for psl in press_slct_btn_list]
    
    # 언론사 이름 : 언론사 클릭 버튼 인 딕셔너리 생성
    press_slct_btn_dict = dict(zip(press_slct_btn_list_nm, press_slct_btn_list))
    
    # 원하는 언론사가 해당 이름 안에 있는 경우
    # 1) 클릭하고
    # 2) 더이상 언론사분류선택 탐색 중지
    if press_nm in press_slct_btn_dict.keys():
        print('<{}> 카테고리에서 <{}>를 찾았으므로 탐색을 종료합니다'.format(press_kind_btn.text, press_nm))
        
        press_slct_btn_dict[press_nm].click()
        time.sleep(sleep_sec)
        
        break



################ 뉴스 크롤링 ########################

print('\n크롤링을 시작합니다.')
# ####동적 제어로 페이지 넘어가며 크롤링
news_dict = {}
idx = 1
cur_page = 1

pbar = tqdm(total=news_num ,leave = True)
    
while idx < news_num:

    table = browser.find_element_by_xpath('//ul[@class="list_news"]')
    li_list = table.find_elements_by_xpath('./li[contains(@id, "sp_nws")]')
    area_list = [li.find_element_by_xpath('.//div[@class="news_area"]') for li in li_list]
    a_list = [area.find_element_by_xpath('.//a[@class="news_tit"]') for area in area_list]
 
    for n in a_list[:min(len(a_list), news_num-idx+1)]:
        n_url = n.get_attribute('href')
        news_dict[idx] = {'title' : n.get_attribute('title'), 
                          'url' : n_url,
                          'text' : crawling_main_text(n_url)}
        
        idx += 1
        pbar.update(1)
        
    if idx < news_num:
        cur_page +=1

        pages = browser.find_element_by_xpath('//div[@class="sc_page_inner"]')
        next_page_url = [p for p in pages.find_elements_by_xpath('.//a') if p.text == str(cur_page)][0].get_attribute('href')

        browser.get(next_page_url)
        time.sleep(sleep_sec)
    else:
        pbar.close()
        
        print('\n브라우저를 종료합니다.\n' + '=' * 100)
        time.sleep(0.7)
        browser.close()
        break

#### 데이터 전처리하기 ###################################################### 

print('데이터프레임 변환\n')
news_df = DataFrame(news_dict).T

folder_path = os.getcwd()
xlsx_file_name = '네이버뉴스_본문_{}개_{}_{}.xlsx'.format(news_num, query, date)

news_df.to_excel(xlsx_file_name)

print('엑셀 저장 완료 | 경로 : {}\\{}\n'.format(folder_path, xlsx_file_name))

os.startfile(folder_path)

print('=' * 100 + '\n결과물의 일부')
news_df

본문 크롤링에 필요한 함수를 로딩하고 있습니다...
----------------------------------------------------------------------------------------------------
검색할 언론사 : JTBC
검색할 키워드  : 코로나
수집 뉴스의 수(숫자만 입력) : 3


브라우저를 실행시킵니다(자동 제어)



C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_19080/3928598573.py:83: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(chrome_path)


설정한 언론사를 선택합니다.



C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_19080/3928598573.py:93: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_opn_btn = browser.find_element_by_xpath('//a[@class="btn_option _search_option_open_btn"]')
C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_19080/3928598573.py:97: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  bx_press = browser.find_element_by_xpath('//div[@role="listbox" and @class="api_group_option_sort _search_option_detail_wrap"]//li[@class="bx press"]')
C:\Users\jhjh3\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:426: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


<방송/통신> 카테고리에서 <JTBC>를 찾았으므로 탐색을 종료합니다

크롤링을 시작합니다.


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_19080/3928598573.py:152: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  table = browser.find_element_by_xpath('//ul[@class="list_news"]')
C:\Users\jhjh3\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:393: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  1.72it/s]



브라우저를 종료합니다.
데이터프레임 변환

엑셀 저장 완료 | 경로 : C:\Users\jhjh3\PJT_02\네이버뉴스_본문_3개_코로나_2022-03-22_11시08분.xlsx

결과물의 일부


,title,url,text
1,"안철수 ""코로나 방역과 소상공인 보상, 인수위 제1민생과제""",https://news.jtbc.joins.com/article/article.as...,인수위원장을 겸직하고 있는 안철수 대통령인수위원회 코로나비상대응특별위원장이 오늘(2...
2,한 번 걸렸으니 슈퍼 항체? '코로나 재감염' 늘고 있다,https://news.jtbc.joins.com/article/article.as...,"[앵커]코로나19 소식입니다. 하루 사이, 38만명 넘는 확진자가 쏟아졌습니다. 며..."
3,코로나에 뜬 밀키트 '못 믿을 속살'…비위생 업체 수두룩,https://news.jtbc.joins.com/article/article.as...,[앵커]집에서 편하게 해먹을 수 있는 간편식을 만드는 업체들이 무더기로 적발됐습니다...


- Summary에 넣을 수 있도록 적용 방법 찾아보기

In [7]:
ti = news_df['title']
co = news_df['text']
print(ti[1])
type(ti)

ti_list = []

for ti_list in news_df['title']:
    ti_list = ti.astype('str')
        
type(ti_list)
type(ti[1])

안철수 "코로나 방역과 소상공인 보상, 인수위 제1민생과제"


str

In [66]:
news_df['title'].toString()

AttributeError: 'Series' object has no attribute 'toString'

### 크롤링한 내용을 요약에 바로 적용해보기

In [10]:
import requests
import json

url = "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize"



document = {"title": news_df.title[1],
            "content": news_df.text[1]}
            
option =  {"language": "ko", 
           "model": "news",
           "tone": 2,
           "summaryCount": 3}

msg = {'document':document, 'option':option}

payload = json.dumps(msg)


headers = {
  'X-NCP-APIGW-API-KEY-ID': 'qj40wl5n0t',
  'X-NCP-APIGW-API-KEY': '1xzXEYgwlsbv9RQxd1onUSByiyvtBDPFV9NEY1mv',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

# r1=response.text  
# r1r1=r1.split('앵커')[1]
# print(r1r1)

{"summary":"코로나19 비상대응특위 회의를 주재하고 있는 안철수 인수위원장 〈사진=연합뉴스〉먼저 안 위원장은 문재인 정부의 코로나19 방역 대응을 '무너진 정치 방역'으로 규정하고 \"과학 방역의 구체적 내용을 세우는 것이 우리 특위의 첫 번째 임무\"라고 강조했습니다.\n안 위원장은 \"손실액 산정을 어떻게 해야 하는지, 보상 방안으로 대출 연장과 세금 감면, 현금 지원에 대해서 어떤 방식으로 접근해야 할지에 대해서 최선의 방법을 찾아야 한다\"고 강조했습니다.\n안 위원장은 \"급하다고 해서 충분한 고려 없이 설익은 해법을 내놓는다면 오히려 혼란과 부작용을 초래할 수 있다\"며 \"향후 다른 감염병 유행 때도 좋은 참고가 될 수 있는 완성도 높은 해법을 국민 앞에 내놓아야 할 것\"이라고 당부했습니다."}


- 앞에 "summary" 빼고 내용만 추출

In [13]:
import requests
import json

url = "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize"

document = {"title": news_df.title[1],
            "content": news_df.text[1]}
            
option =  {"language": "ko", 
           "model": "news",
           "tone": 2,
           "summaryCount": 3}

msg = {'document':document, 'option':option}

payload = json.dumps(msg)


headers = {
  'X-NCP-APIGW-API-KEY-ID': 'qj40wl5n0t',
  'X-NCP-APIGW-API-KEY': '1xzXEYgwlsbv9RQxd1onUSByiyvtBDPFV9NEY1mv',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

# print(response.text)

r1=response.text  
r1r1=r1.split(':')[1]
print(r1r1)

"코로나19 비상대응특위 회의를 주재하고 있는 안철수 인수위원장 〈사진=연합뉴스〉먼저 안 위원장은 문재인 정부의 코로나19 방역 대응을 '무너진 정치 방역'으로 규정하고 \"과학 방역의 구체적 내용을 세우는 것이 우리 특위의 첫 번째 임무\"라고 강조했습니다.\n안 위원장은 \"손실액 산정을 어떻게 해야 하는지, 보상 방안으로 대출 연장과 세금 감면, 현금 지원에 대해서 어떤 방식으로 접근해야 할지에 대해서 최선의 방법을 찾아야 한다\"고 강조했습니다.\n안 위원장은 \"급하다고 해서 충분한 고려 없이 설익은 해법을 내놓는다면 오히려 혼란과 부작용을 초래할 수 있다\"며 \"향후 다른 감염병 유행 때도 좋은 참고가 될 수 있는 완성도 높은 해법을 국민 앞에 내놓아야 할 것\"이라고 당부했습니다."}


In [123]:
import requests
import json

url = "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize"

document = {"title": news_df.title[1],
            "content": news_df.text[1]}
            
option =  {"language": "ko", 
           "model": "news",
           "tone": 2,
           "summaryCount": 3}

msg = {'document':document, 'option':option}

payload = json.dumps(msg)


headers = {
  'X-NCP-APIGW-API-KEY-ID': 'qj40wl5n0t',
  'X-NCP-APIGW-API-KEY': '1xzXEYgwlsbv9RQxd1onUSByiyvtBDPFV9NEY1mv',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

# print(response.text)

r1=response.text
r1 = json.loads(r1)
r1r1=r1["summary"]
print(r1r1)

코로나19 비상대응특위 회의를 주재하고 있는 안철수 인수위원장 〈사진=연합뉴스〉먼저 안 위원장은 문재인 정부의 코로나19 방역 대응을 '무너진 정치 방역'으로 규정하고 "과학 방역의 구체적 내용을 세우는 것이 우리 특위의 첫 번째 임무"라고 강조했습니다.
안 위원장은 "손실액 산정을 어떻게 해야 하는지, 보상 방안으로 대출 연장과 세금 감면, 현금 지원에 대해서 어떤 방식으로 접근해야 할지에 대해서 최선의 방법을 찾아야 한다"고 강조했습니다.
안 위원장은 "급하다고 해서 충분한 고려 없이 설익은 해법을 내놓는다면 오히려 혼란과 부작용을 초래할 수 있다"며 "향후 다른 감염병 유행 때도 좋은 참고가 될 수 있는 완성도 높은 해법을 국민 앞에 내놓아야 할 것"이라고 당부했습니다.


- 요약한것 읽어주기

In [124]:
import os
import sys
import urllib.request
client_id = "qj40wl5n0t"
client_secret = "1xzXEYgwlsbv9RQxd1onUSByiyvtBDPFV9NEY1mv"
encText = urllib.parse.quote(r1r1)
data = "speaker=njihun&volume=0&speed=0&pitch=0&format=mp3&text=" + encText;
url = "https://naveropenapi.apigw.ntruss.com/tts-premium/v1/tts"
request = urllib.request.Request(url)
request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
request.add_header("X-NCP-APIGW-API-KEY",client_secret)
response = urllib.request.urlopen(request, data=data.encode('utf-8'))
rescode = response.getcode()
if(rescode==200):
    print("TTS mp3 저장")
    response_body = response.read()
    with open('1113.mp3', 'wb') as f:
        f.write(response_body)
else:
    print("Error Code:" + rescode)

TTS mp3 저장


### 본격 크롤링한 뉴스 리스트를 요약리스트에 넣어서 결과 도출하기

In [125]:
len(news_df.title)

import os
import sys
import requests
import json
import urllib.request

client_id = "qj40wl5n0t"
client_secret = "1xzXEYgwlsbv9RQxd1onUSByiyvtBDPFV9NEY1mv"

for i in range(1, len(news_df.title)+1):
    headers = {
    "X-NCP-APIGW-API-KEY-ID": client_id,
    "X-NCP-APIGW-API-KEY": client_secret,
    "Content-Type": "application/json"
}
    language = "ko" # Language of document (ko, ja )
    model = "news" # Model used for summaries (general, news)
    tone = "2" # Converts the tone of the summarized result. (0, 1, 2, 3)
    summaryCount = "3" # This is the number of sentences for the summarized document.
    url= "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize" 
    title= ""
    content = news_df.text[i]
    data = {
        "document": {
        "title": title,
        "content" : content
        },
        "option": {
        "language": language,
        "model": model,
        "tone": tone,
        "summaryCount" : summaryCount
        }
    }
    # print(json.dumps(data, indent=4, sort_keys=True))
    response = requests.post(url, data=json.dumps(data), headers=headers)
    rescode = response.status_code
    if(rescode == 200):
        r1=response.text  
        # print(r1)

    else:
        print("Error : " + response.text)
        
    r1 = json.loads(r1)
    r1r1=r1["summary"]
    print(r1r1)



인수위원장을 겸직하고 있는 안철수 대통령인수위원회 코로나비상대응특별위원장이 오늘(21일) 통의동 인수위원회에서 코로나비상대응특별위원회 제1차 회의를 열었습니다.
코로나19 비상대응특위 회의를 주재하고 있는 안철수 인수위원장 〈사진=연합뉴스〉먼저 안 위원장은 문재인 정부의 코로나19 방역 대응을 '무너진 정치 방역'으로 규정하고 "과학 방역의 구체적 내용을 세우는 것이 우리 특위의 첫 번째 임무"라고 강조했습니다.
이어 문재인 정부에서 시행했던 방역 정책들을 점검하고 무엇이 잘못됐는지 확인하는 일을 첫 과제로 꼽았습니다.
하루 사이, 38만명 넘는 확진자가 쏟아졌습니다.
전문가들은 코로나에 다시 걸리지 않는다는 이른바 '슈퍼 항체 보유자'는 없다고 봐야 한다며 완치자도 방역수칙을 철저히 지켜야 한다고 강조합니다.[천은미/이화여대 목동병원 호흡기내과 교수 : 기존에 델타 변이까지 감염됐던 분이면 굉장히 조심합니다.
오미크론은 감염이 될 수 있고.]특히 오미크론 변이의 하위 계통인 이른바 스텔스 오미크론 BA2변이의 등장도 재감염 사례를 악화시키는 요인일수 있습니다.
앵커]집에서 편하게 해먹을 수 있는 간편식을 만드는 업체들이 무더기로 적발됐습니다.
플라스틱 바구니 안엔 진공포장된 각종 고기가 가득 쌓여있습니다.
냉동보관 해야 할 고기를 냉장창고에 뒀다가 적발된 겁니다.[축산물가공업소 관계자 : 이게 원료육이어서…원료 냉동…]마트 안에 있는 정육점입니다.


## 뉴스 사이사이 구분 넣어주기 (ex) 1번째 뉴스입니다.)

In [126]:
len(news_df.title)
news_list=[]

import os
import sys
import requests
import json
import urllib.request

client_id = "qj40wl5n0t"
client_secret = "1xzXEYgwlsbv9RQxd1onUSByiyvtBDPFV9NEY1mv"

for i in range(1, len(news_df.title)+1):
    headers = {
    "X-NCP-APIGW-API-KEY-ID": client_id,
    "X-NCP-APIGW-API-KEY": client_secret,
    "Content-Type": "application/json"
}
    language = "ko" # Language of document (ko, ja )
    model = "news" # Model used for summaries (general, news)
    tone = "2" # Converts the tone of the summarized result. (0, 1, 2, 3)
    summaryCount = "3" # This is the number of sentences for the summarized document.
    url= "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize" 
    title= ""
    #print([i],"번째 뉴스입니다.")
    content = news_df.text[i]
    data = {
        "document": {
        "title": title,
        "content" : content
        },
        "option": {
        "language": language,
        "model": model,
        "tone": tone,
        "summaryCount" : summaryCount
        }
    }
    # print(json.dumps(data, indent=4, sort_keys=True))
    response = requests.post(url, data=json.dumps(data), headers=headers)
    rescode = response.status_code
    if(rescode == 200):
        r1=response.text  
        # print(r1)

    else:
        print("Error : " + response.text)
        
    r1 = json.loads(r1)
    r1r1=r1["summary"]
    news_list.append('{}번째 뉴스입니다.'.format(i))
    news_list.append(r1r1)
    # print(r1r1)

print(news_list)
#type(news_list)


['1번째 뉴스입니다.', '인수위원장을 겸직하고 있는 안철수 대통령인수위원회 코로나비상대응특별위원장이 오늘(21일) 통의동 인수위원회에서 코로나비상대응특별위원회 제1차 회의를 열었습니다.\n코로나19 비상대응특위 회의를 주재하고 있는 안철수 인수위원장 〈사진=연합뉴스〉먼저 안 위원장은 문재인 정부의 코로나19 방역 대응을 \'무너진 정치 방역\'으로 규정하고 "과학 방역의 구체적 내용을 세우는 것이 우리 특위의 첫 번째 임무"라고 강조했습니다.\n이어 문재인 정부에서 시행했던 방역 정책들을 점검하고 무엇이 잘못됐는지 확인하는 일을 첫 과제로 꼽았습니다.', '2번째 뉴스입니다.', "하루 사이, 38만명 넘는 확진자가 쏟아졌습니다.\n전문가들은 코로나에 다시 걸리지 않는다는 이른바 '슈퍼 항체 보유자'는 없다고 봐야 한다며 완치자도 방역수칙을 철저히 지켜야 한다고 강조합니다.[천은미/이화여대 목동병원 호흡기내과 교수 : 기존에 델타 변이까지 감염됐던 분이면 굉장히 조심합니다.\n오미크론은 감염이 될 수 있고.]특히 오미크론 변이의 하위 계통인 이른바 스텔스 오미크론 BA2변이의 등장도 재감염 사례를 악화시키는 요인일수 있습니다.", '3번째 뉴스입니다.', '앵커]집에서 편하게 해먹을 수 있는 간편식을 만드는 업체들이 무더기로 적발됐습니다.\n플라스틱 바구니 안엔 진공포장된 각종 고기가 가득 쌓여있습니다.\n냉동보관 해야 할 고기를 냉장창고에 뒀다가 적발된 겁니다.[축산물가공업소 관계자 : 이게 원료육이어서…원료 냉동…]마트 안에 있는 정육점입니다.']


In [127]:
r1

{'summary': '앵커]집에서 편하게 해먹을 수 있는 간편식을 만드는 업체들이 무더기로 적발됐습니다.\n플라스틱 바구니 안엔 진공포장된 각종 고기가 가득 쌓여있습니다.\n냉동보관 해야 할 고기를 냉장창고에 뒀다가 적발된 겁니다.[축산물가공업소 관계자 : 이게 원료육이어서…원료 냉동…]마트 안에 있는 정육점입니다.'}

In [128]:
news_list

['1번째 뉴스입니다.',
 '인수위원장을 겸직하고 있는 안철수 대통령인수위원회 코로나비상대응특별위원장이 오늘(21일) 통의동 인수위원회에서 코로나비상대응특별위원회 제1차 회의를 열었습니다.\n코로나19 비상대응특위 회의를 주재하고 있는 안철수 인수위원장 〈사진=연합뉴스〉먼저 안 위원장은 문재인 정부의 코로나19 방역 대응을 \'무너진 정치 방역\'으로 규정하고 "과학 방역의 구체적 내용을 세우는 것이 우리 특위의 첫 번째 임무"라고 강조했습니다.\n이어 문재인 정부에서 시행했던 방역 정책들을 점검하고 무엇이 잘못됐는지 확인하는 일을 첫 과제로 꼽았습니다.',
 '2번째 뉴스입니다.',
 "하루 사이, 38만명 넘는 확진자가 쏟아졌습니다.\n전문가들은 코로나에 다시 걸리지 않는다는 이른바 '슈퍼 항체 보유자'는 없다고 봐야 한다며 완치자도 방역수칙을 철저히 지켜야 한다고 강조합니다.[천은미/이화여대 목동병원 호흡기내과 교수 : 기존에 델타 변이까지 감염됐던 분이면 굉장히 조심합니다.\n오미크론은 감염이 될 수 있고.]특히 오미크론 변이의 하위 계통인 이른바 스텔스 오미크론 BA2변이의 등장도 재감염 사례를 악화시키는 요인일수 있습니다.",
 '3번째 뉴스입니다.',
 '앵커]집에서 편하게 해먹을 수 있는 간편식을 만드는 업체들이 무더기로 적발됐습니다.\n플라스틱 바구니 안엔 진공포장된 각종 고기가 가득 쌓여있습니다.\n냉동보관 해야 할 고기를 냉장창고에 뒀다가 적발된 겁니다.[축산물가공업소 관계자 : 이게 원료육이어서…원료 냉동…]마트 안에 있는 정육점입니다.']

In [129]:
len(news_df.title)

3

In [130]:
news_df.title[1]

'안철수 "코로나 방역과 소상공인 보상, 인수위 제1민생과제"'

In [131]:
r1r1

'앵커]집에서 편하게 해먹을 수 있는 간편식을 만드는 업체들이 무더기로 적발됐습니다.\n플라스틱 바구니 안엔 진공포장된 각종 고기가 가득 쌓여있습니다.\n냉동보관 해야 할 고기를 냉장창고에 뒀다가 적발된 겁니다.[축산물가공업소 관계자 : 이게 원료육이어서…원료 냉동…]마트 안에 있는 정육점입니다.'

#### 구분한 뉴스를 음성에 넣으려했는데 안된다,, 더 연구해 보자

In [133]:
    encText = urllib.parse.quote(news_list)
    data = "speaker=nara&volume=0&speed=0&pitch=0&format=mp3&text=" + encText;
    url = "https://naveropenapi.apigw.ntruss.com/tts-premium/v1/tts"
    request = urllib.request.Request(url)
    request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
    request.add_header("X-NCP-APIGW-API-KEY",client_secret)
    response = urllib.request.urlopen(request, data=data.encode('utf-8'))
    rescode = response.getcode()
    if(rescode==200):
        print("TTS mp3 저장")
        response_body = response.read()
        with open('1117.mp3', 'wb') as f:
            f.write(response_body)
    else:
        print("Error Code:" + rescode)

TypeError: quote_from_bytes() expected bytes

#### 딕셔너리 즉 news_list[1] 하나를 넣으면 되는데 한번에는 안되네?

In [87]:
import os
import sys
import urllib.request
client_id = "qj40wl5n0t"
client_secret = "1xzXEYgwlsbv9RQxd1onUSByiyvtBDPFV9NEY1mv"
encText = urllib.parse.quote(news_list[1])
data = "speaker=njinho&volume=0&speed=0&pitch=0&format=mp3&text=" + encText;
url = "https://naveropenapi.apigw.ntruss.com/tts-premium/v1/tts"
request = urllib.request.Request(url)
request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
request.add_header("X-NCP-APIGW-API-KEY",client_secret)
response = urllib.request.urlopen(request, data=data.encode('utf-8'))
rescode = response.getcode()
if(rescode==200):
    print("TTS mp3 저장")
    response_body = response.read()
    with open('1115.mp3', 'wb') as f:
        f.write(response_body)
else:
    print("Error Code:" + rescode)

TTS mp3 저장


## 그럼 음성 파일을 각각 만들어 보자

In [121]:
len(news_df.title)
# news_list = []

import os
import sys
import requests
import json
import urllib.request

client_id = "qj40wl5n0t"
client_secret = "1xzXEYgwlsbv9RQxd1onUSByiyvtBDPFV9NEY1mv"

for i in range(len(news_df.title)):
    headers = {
    "X-NCP-APIGW-API-KEY-ID": client_id,
    "X-NCP-APIGW-API-KEY": client_secret,
    "Content-Type": "application/json"
}
    language = "ko" # Language of document (ko, ja )
    model = "news" # Model used for summaries (general, news)
    tone = "2" # Converts the tone of the summarized result. (0, 1, 2, 3)
    summaryCount = "3" # This is the number of sentences for the summarized document.
    url= "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize" 
    title= ""
    content = news_df.text[i+1]
    data = {
        "document": {
        "title": title,
        "content" : content
        },
        "option": {
        "language": language,
        "model": model,
        "tone": tone,
        "summaryCount" : summaryCount
        }
    }
    # print(json.dumps(data, indent=4, sort_keys=True))
    response = requests.post(url, data=json.dumps(data), headers=headers)
    rescode = response.status_code
    if(rescode == 200):
        r1=response.text  
        # print(r1)

    else:
        print("Error : " + response.text)

    r1 = json.loads(r1)
    r1r1= r1['summary']
    r1r1 = '{}번째 뉴스입니다.{}'.format(i+1, r1r1)
    

#print(news_list)
#type(news_list)

    ## 읽어줘요

    encText = urllib.parse.quote(r1r1)
    data = "speaker=njihun&volume=0&speed=0&pitch=0&format=mp3&text=" + encText;
    url = "https://naveropenapi.apigw.ntruss.com/tts-premium/v1/tts"
    request = urllib.request.Request(url)
    request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
    request.add_header("X-NCP-APIGW-API-KEY",client_secret)
    response = urllib.request.urlopen(request, data=data.encode('utf-8'))
    rescode = response.getcode()
    if(rescode==200):
        print("TTS mp3 저장")
        response_body = response.read()
        fn = 'voice{}.mp3'.format(i)
        with open(fn, 'wb') as f:
            f.write(response_body)
    else:
        print("Error Code:" + rescode)

TTS mp3 저장
TTS mp3 저장
TTS mp3 저장
